In [67]:
#distance square
def dist_sq(a, b):
    return np.sum((a-b)**2)
#minimum distance square for every point to the centroid
def point_sq(data,centroid):
    dist=[]
    for i in range(data.shape[0]):
        dist.append(min(dist_sq(data.ix[i],c) for c in centroid))
    return dist
        
#calculate probability
def dist_prob(Dist):
    return Dist/np.sum(Dist)

In [102]:
def kmeansplusplus(data, k, d):
    index=int(np.random.choice(data.shape[0],1))
    centroid=np.array(data.ix[index])
    for number in range(k-1):
        distance=point_sq(data,np.array(centroid))
        prob=dist_prob(distance).tolist()
        index=int(np.random.choice(data.shape[0],1,prob))
        centroid=np.vstack([centroid,np.array(data.ix[index])])
    return centroid


In [103]:
def kmeans(dataSet, initial):
    centroids=np.array(initial)
    # Initialize book keeping vars.
    iterations = 0
    oldCentroids = np.zeros([initial.shape[0],initial.shape[1]])
    
    # Run the main k-means algorithm
    while not shouldStop(oldCentroids, centroids, iterations):
        # Save old centroids for convergence test. Book keeping.
        oldCentroids = centroids
        iterations += 1
        
        # Assign labels to each datapoint based on centroids
        l= getLabels(dataSet, centroids)
        
        # Assign centroids based on datapoint labels
        centroids = getCentroids(dataSet, l)
        
    # We can get the labels too by calling getLabels(dataSet, centroids)
    return centroids, np.array(l)


In [104]:
# Function: Should Stop
# -------------
# Returns True or False if k-means is done. K-means terminates either
# because it has run a maximum number of iterations OR the centroids
# stop changing.
def shouldStop(oldCentroids, centroids, iterations):
    if iterations > 50: return True
    return oldCentroids.all == centroids.all


In [105]:
# Function: Get Labels
# -------------
# Returns a label for each piece of data in the dataset. 
def getLabels(dataSet, centroids):
    # For each element in the dataset, chose the closest centroid. 
    # Make that centroid the element's label.
    l=[]
    for i in range(data.shape[0]):
        l.append(np.argmin(list(dist_sq(data.ix[i],c) for c in centroids)))
    return l

            


In [106]:
# Function: Get Centroids
# -------------
# Returns k random centroids, each of dimension n.
def getCentroids(dataSet, labels):
    # Each centroid is the geometric mean of the points that
    # have that centroid's label. Important: If a centroid is empty (no points have
    # that centroid's label) you should randomly re-initialize it.
    data_new = dataSet.copy()
    data_new['Labels'] = labels
    data_new = data_new.groupby(['Labels']).mean().iloc[:,:2]
    return np.array(data_new)
    

In [107]:
#test 
data=DataFrame([np.random.random(100),np.random.random(100)]).transpose()
initial= kmeansplusplus(data,8,2)

In [108]:
kmeans(data, initial)

(array([[ 0.23801647,  0.52264777],
        [ 0.70476309,  0.51468006],
        [ 0.90115282,  0.17373722],
        [ 0.51772984,  0.75059068],
        [ 0.6232516 ,  0.25712429],
        [ 0.88844844,  0.77337313],
        [ 0.14159695,  0.24405234],
        [ 0.24916151,  0.90676037]]),
 array([4, 2, 4, 6, 4, 4, 5, 6, 2, 0, 6, 6, 4, 6, 5, 0, 3, 0, 6, 4, 5, 1, 4,
        1, 1, 6, 0, 2, 0, 3, 3, 5, 3, 3, 2, 1, 0, 1, 3, 6, 2, 5, 2, 1, 6, 4,
        0, 0, 4, 7, 7, 0, 7, 1, 6, 1, 1, 2, 2, 5, 0, 4, 0, 5, 7, 4, 4, 7, 0,
        6, 6, 3, 0, 3, 6, 2, 6, 0, 7, 1, 7, 3, 1, 3, 4, 4, 0, 3, 7, 6, 5, 7,
        0, 3, 1, 4, 6, 6, 6, 6]))